In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/60k-stack-overflow-questions-with-quality-rate/train.csv')
valid = pd.read_csv('/kaggle/input/60k-stack-overflow-questions-with-quality-rate/valid.csv')

In [ ]:
train.head()

In [ ]:
# let's see the target variable 
train.Y.value_counts()

In [ ]:
# the dataset is balanced 
# checking for the null values present in the dataset

pd.DataFrame(train.isnull().sum())

In [ ]:
pd.DataFrame(train.isna().sum())

In [ ]:
# importing nlp libraries 

import nltk 
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('punkt')

import string
def text_process(text):
    text = text.translate(str.maketrans(" ", " ", string.punctuation))
    
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

In [ ]:
text = train['Body'][0]
print('Given text',text)
print('\nProcessed text',text_process(text))

In [ ]:
train['text'] = train['Title'] + " " + train['Body'] + " " + train['Tags']

In [ ]:
text_feat = train.text.apply(text_process)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer("english")
features = vectorizer.fit_transform(text_feat)
features

In [ ]:
def changing_labels(text):
    if text == "HQ":
        return 0
    if text == "LQ_EDIT":
        return 1 
    if text == "LQ_CLOSE":
        return -1

In [ ]:
train['labels'] = train['Y'].apply(changing_labels)

In [ ]:
train['labels'].value_counts()

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression


model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, features, train['labels'], scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
# report the model performance
import matplotlib.pyplot as plt 
plt.plot(n_scores)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model_mnb = MultinomialNB(alpha=0.2)

In [ ]:
model_lr.fit(features, train['labels'])
model_mnb.fit(features, train['labels'])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

def conf_matrix(y_true, y_pred):
    print(confusion_matrix(y_true, y_pred))
    print(accuracy_score(y_true, y_pred))

In [ ]:
valid['text'] = valid['Title'] + " " + valid['Body'] + " " + valid['Tags']
text_feat = valid.text.apply(text_process)
vectorizer.transform(text_feat)
valid['labels'] = valid['Y'].apply(changing_labels)

In [ ]:
features = vectorizer.transform(text_feat)

y_pred_lr = model_lr.predict(features)
y_pred_mnb = model_mnb.predict(features)

In [ ]:
conf_matrix(valid['labels'] , y_pred_lr)
conf_matrix(valid['labels'] , y_pred_mnb)